Reference : https://sefiks.com/2019/02/13/apparent-age-and-gender-prediction-in-keras/

google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 학습 데이터 나이대별 경로 저장

학습데이터는 AFAD dataset, 0-9세, 40~100세까지는 추가적으로 수집

In [ ]:
import os
import pickle

# 이미지 경로가 포함된 디렉토리 설정
image_dir = '/content/drive/MyDrive/AFAD-Full/'

# 나이대별로 이미지 파일 경로 리스트 생성
age_labels = ['00-09', '10-19', '20-29', '30-39', '40-49', '50-59', '60-100']
age_ranges = [(0, 9), (10, 19), (20, 29), (30, 39), (40, 49), (50, 59), (60, 100)]
age_image_paths = {label: [] for label in age_labels}

# 디렉토리 구조를 탐색하여 파일 경로 리스트를 생성
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith(('jpg', 'jpeg', 'png')):
            # 디렉토리 이름에서 나이를 추출 (예: '/path/to/40/111/file.jpg'에서 40 추출)
            try:
                age = int(os.path.basename(os.path.dirname(root)))
                for label, (age_min, age_max) in zip(age_labels, age_ranges):
                    if age_min <= age <= age_max:
                        age_image_paths[label].append(os.path.join(root, file))
                        break
            except ValueError:
                continue  # 나이 추출 실패한 경우 무시

# 나이대별 경로를 파일로 저장 (Google Drive에 저장)
for label in age_labels:
    file_path = f'/content/drive/MyDrive/{label}_image_paths.pkl'
    with open(file_path, 'wb') as f:
        pickle.dump(age_image_paths[label], f)
    print(f"{label} 구간에 대해 총 {len(age_image_paths[label])}개의 이미지 경로가 저장되었습니다.")

print("모든 나이대에 대한 이미지 경로가 저장되었습니다.")

# 나이대별 학습 데이터 불러오기

In [ ]:
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, ZeroPadding2D, Conv2D, MaxPooling2D, Flatten, Dropout, Activation
import pickle
import re

#  이미지 크기 설정
target_size = (224, 224)

# 이미지 픽셀 데이터를 가져오는 함수
def getImagePixels(image_path):
    img = image.load_img(image_path, color_mode='rgb', target_size=target_size)
    x = image.img_to_array(img).reshape(1, -1)[0]
    return x
# 나이대별로 이미지 파일 경로 리스트 생성
age_labels = ['00-09', '10-19', '20-29', '30-39', '40-49', '50-59', '60-100']
age_ranges = [(0, 9), (10, 19), (20, 29), (30, 39), (40, 49), (50, 59), (60, 100)]
age_image_paths = {label: [] for label in age_labels}

# 각 파일당 500개씩 불러오기
sampled_image_paths = []
for label in age_labels:
    file_path = f'/content/drive/MyDrive/{label}_image_paths.pkl'
    with open(file_path, 'rb') as f:
        paths = pickle.load(f)
        sampled_image_paths.extend(paths[:500])  # 각 파일당 1000개씩 불러오기

# 파일 경로에서 나이 정보를 추출하는 함수
def extract_age_from_path(path):
    match = re.search(r'/(\d+)/', path)
    if match:
        return int(match.group(1))
    else:
        return None

# 파일 경로에서 성별 정보를 추출하는 함수
def extract_gender_from_path(path):
    match = re.search(r'/(111|112)/', path)
    if match:
        return 0 if match.group(1) == '111' else 1
    else:
        return None

# 나이 및 성별 정보 추출
ages = [extract_age_from_path(path) for path in sampled_image_paths]
genders = [extract_gender_from_path(path) for path in sampled_image_paths]

# 데이터프레임 생성
data = {
    'full_path': sampled_image_paths,
    'gender': genders,  # 경로에서 추출한 성별 추가
    'age': ages  # 경로에서 추출한 나이 추가
}

df = pd.DataFrame(data)

# 나이 그룹화
age_bins = [0, 9, 19, 29, 39, 49, 59, 100]
age_labels = ['00-09', '10-19', '20-29', '30-39', '40-49', '50-59', '60-100']
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=True, include_lowest=True)

# 'pixels' 열을 빈 리스트로 초기화
df['pixels'] = None

# 픽셀 데이터를 DataFrame에 추가
for idx, row in df.iterrows():
    print(f"Processing {idx+1}/{len(df)}: {row['full_path']}")
    pixel_data = getImagePixels(row['full_path'])
    if pixel_data is not None:
        df.at[idx, 'pixels'] = pixel_data.tolist()  # 리스트 형태로 저장

# 나이 그룹에 대한 클래스를 생성
age_group_to_class = {label: i for i, label in enumerate(age_labels)}
df['age_class'] = df['age_group'].map(age_group_to_class)

# NaN 값을 가진 행 제거
df = df.dropna(subset=['age_class'])

# Keras에서 사용할 age_class 정보의 One-hot encoding
classes = len(age_labels)  # 나이 그룹의 개수
target = df['age_class'].values
target_classes = to_categorical(target, classes)

# print(df)

#  훈련 데이터와 테스트 데이터로 분할

In [ ]:
features = []

for i in range(0, df.shape[0]):
    features.append(df['pixels'].values[i])

features = np.array(features)
features = features.reshape(features.shape[0], 224, 224, 3)

train_x, test_x, train_y, test_y = train_test_split(features, target_classes, test_size=0.30)

# 모델 학습

In [ ]:
# VGG-Face model
# 모델 입력 정의
input_tensor = Input(shape=(224, 224, 3))

# VGG-Face 모델 정의
x = ZeroPadding2D((1, 1))(input_tensor)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

x = ZeroPadding2D((1, 1))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

x = ZeroPadding2D((1, 1))(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

x = ZeroPadding2D((1, 1))(x)
x = Conv2D(512, (3, 3), activation='relu')(x)
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(512, (3, 3), activation='relu')(x)
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(512, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

x = ZeroPadding2D((1, 1))(x)
x = Conv2D(512, (3, 3), activation='relu')(x)
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(512, (3, 3), activation='relu')(x)
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(512, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

x = Conv2D(4096, (7, 7), activation='relu')(x)
x = Dropout(0.5)(x)
x = Conv2D(4096, (1, 1), activation='relu')(x)
x = Dropout(0.5)(x)
x = Conv2D(2622, (1, 1))(x)
x = Flatten()(x)
x = Activation('softmax')(x)

model = Model(inputs=input_tensor, outputs=x)

# Pre-trained weights of VGG-Face model.
model.load_weights('/content/drive/MyDrive/vgg_face_weights.h5')

# 모델의 마지막 7개 레이어를 제외하고 나머지를 고정
for layer in model.layers[:-7]:
    layer.trainable = False

# 모델 수정 (출력 레이어의 노드 수를 7로 설정)
x = Conv2D(7, (1, 1), name='predictions')(model.layers[-4].output)
x = Flatten()(x)
x = Activation('softmax')(x)

age_model = Model(inputs=model.input, outputs=x)

age_model.load_weights('/content/drive/MyDrive/age_model.keras')



# 모델 컴파일
age_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# 체크포인트 설정
checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/age_model.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
####################################################################################


scores = []
epochs = 200
batch_size = 64

for i in range(epochs):
    print("epoch ", i)

    ix_train = np.random.choice(train_x.shape[0], size=batch_size)

    score = age_model.fit(train_x[ix_train], train_y[ix_train],
                          epochs=1, validation_data=(test_x, test_y), callbacks=[checkpointer])

    scores.append(score)

# 모델 평가
loss, accuracy = 	age_model.evaluate(test_x, test_y, verbose=1)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")